In [3]:
import pandas as pd
from datetime import timedelta


In [4]:
df = pd.read_csv("../data/raw_data.csv")

C:\Users\metal\AppData\Local\Temp\ipykernel_93424\3407229622.py:1: DtypeWarning: Columns (28,30,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/raw_data.csv")


In [5]:
def get_fx_date(launched_date):
    """Return the date to use for FX rate lookup.

    Mon–Fri: use the same day.
    Sat:     use Friday (one day earlier).
    Sun:     use Friday (two days earlier).
    """
    date = pd.to_datetime(launched_date)

    weekday = date.weekday() # 0=Mon ... 4=Fri, 5=Sat, 6=Sun

    if weekday == 5: # Saturday
        date = date - timedelta(days=1)
    elif weekday == 6: # Sunday
        date = date - timedelta(days=2)

    return date.normalize() 

def add_fx_mean(df: pd.DataFrame, fx_folder: str = "../data/fx_rates") -> pd.DataFrame:
    """Add fx_daily_mean for each row using daily FX CSVs and weekend logic.

    - launched_at is assumed to be epoch seconds.
    - For non-USD rows, FX rate is looked up from <currency>-USD_daily.csv
      using an FX date where Sat/Sun are mapped back to Friday.
    - For USD rows, fx_daily_mean is set to 1.0.

    Parameters
    ----------
    df : pd.DataFrame
        Project-level dataset with at least columns: 'currency', 'launched_at'.
    fx_folder : str, optional
        Folder path where the FX CSV files are stored.

    Returns
    -------
    pd.DataFrame
        Copy of the input DataFrame with new columns:
        - launched_date
        - fx_date
        - fx_daily_mean
    """
    
    df = df.copy()
    df["launched_date"] = pd.to_datetime(df["launched_at"], unit="s").dt.floor("D")
    
    # Luodaan fx_date: viikonloput -> perjantai
    df["fx_date"] = df["launched_date"].apply(get_fx_date)
    
    # Separate currencies to USD & others
    df_usd = df[df["currency"] == "USD"].copy()
    df_non_usd = df[df["currency"] != "USD"].copy()

    # USD - USD uses static 1.0 fx-rate
    df_usd["fx_daily_mean"] = 1.0
    
    # Get list of non-USD currencies
    currency_list = set(df_non_usd["currency"])

    merged_parts = []

    for currency in currency_list:
        df_sub = df_non_usd[df_non_usd["currency"] == currency].copy()

        # Get fx-rates from csv
        fx_path = f"{fx_folder}/{currency}-USD_daily.csv"
        fx = pd.read_csv(fx_path)

        fx["fx_date"] = pd.to_datetime(fx["timestamp"]).dt.floor("D")

        df_sub = df_sub.merge(
            fx[["fx_date", "fx_daily_mean"]],
            on = "fx_date",
            how = "left",
        )
        
        merged_parts.append(df_sub)

    df_final = pd.concat([df_usd] + merged_parts, ignore_index=True)

    return df_final 
    



In [6]:
def fix_dates(df: pd.DataFrame, avg_duration_days: int = 34) -> pd.DataFrame:
    """
    Fix launched_at and deadline values:
    - Drop rows where deadline == 0 (these also have launched_at == 0).
    - For rows where launched_at == 0 but deadline is valid,
      impute launched_at as deadline - avg_duration_days (in Unix time).

    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame containing 'launched_at' and 'deadline' columns
        in Unix timestamp (seconds).
    avg_duration_days : int, optional
        Average project duration in days, used for imputing missing
        launched_at values. Default is 34.

    Returns
    -------
    pd.DataFrame
        Cleaned DataFrame with:
        - rows with deadline == 0 removed
        - imputed launched_at where applicable
    """
    df = df.copy()

    # Remove lines with 'deadline' == 0 (these also have 'launched_at' == 0)
    df = df[df['deadline'] != 0]

    # Count the seconds from avg_duration_days for Unix impution
    seconds_per_day = 24 * 60 * 60
    offset_seconds = avg_duration_days * seconds_per_day

    # Imputate 'launched_at' where 'launched_at' == 0 but 'deadline' != 0
    mask_launched_missing = (df['launched_at'] == 0) & (df['deadline'] != 0)
    df.loc[mask_launched_missing, 'launched_at'] = (
        df.loc[mask_launched_missing, 'deadline'] - offset_seconds
    )

    # Return fixed dataframe
    return df

In [7]:
def drop_unused_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Remove ucnessary columns from the dataset:

    - backers_count
    - percent_funded
    - all '*pledged*' columns
    - spotlight
    - staff_pick
    - all *profile* columns
    - all *creator* columns (except 'creator_id')
    - all *URL* columns
    - category_name
    - category_id
    - collected_at
    - state_changed_at
    - id
    - usd_type
    - all *location* columns
    - all currency exchange rates 

    Parameters
    ----------
    df : pd.DataFrame
        Input Dataframe with original dataset columns.

    Returns
    -------
    df : pd.DataFrame
        An updated Dataframe with the unnecessary columns removed.
    """
    df = df.copy()

    cols_to_drop = [
        "backers_count", "percent_funded",
        "converted_pledged_amount", "pledged", "usd_pledged",
        "spotlight", "staff_pick",
        "profile_blurb", "profile_id", "profile_name",
        "profile_project_id", "profile_state", "profile_state_changed_at",
        "creator_name", "creator_url",
        "category_url", "project_url",
        "category_name", "category_id",
        "collected_at", "state_changed_at",
        "id", "usd_type", "location_country", "location_id", 
        "location_name", "location_state", "location_type",
        "static_usd_rate", "usd_exchange_rate", "fx_rate"
    ]

    # Drop only the columns found in the dataframe
    existing_cols = [c for c in cols_to_drop if c in df.columns]
    df = df.drop(columns=existing_cols)

    return df

In [8]:
def merge_categories(df: pd.DataFrame) -> pd.DataFrame:
    """
    Normalize category parent names by converting 'Journalism' and 'Dance' to 'Others'.

    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame containing a 'category_parent_name' column.
    
    Returns
    -------
    pd.DataFrame
        A new DataFrame with the corrected 'category_parent_name' values.
    """
    df = df.copy()
    df['category_parent_name'] = df['category_parent_name'].replace({
        'Journalism': 'Others',
        'Dance': 'Others'
    })
    return df

In [9]:
def merge_comics(df: pd.DataFrame) -> pd.DataFrame:
    """
    Normalize category parent names by converting 'comics' to 'Comics'.

    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame containing a 'category_parent_name' column.

    Returns
    -------
    pd.DataFrame
        A new DataFrame with the corrected 'category_parent_name' values.
    """
    df = df.copy()
    df['category_parent_name'] = df['category_parent_name'].replace({
        'comics': 'Comics'
    })
    return df

In [10]:
def calculate_usd_goal(df) -> pd.DataFrame:
    df = df.copy()
    df = add_fx_mean(df) 
    df["usd_goal_fx"] = (df["goal"] * df["fx_daily_mean"]).round()

    return df

In [11]:
def clean_state(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean and binarize the 'state' column:

    - Treat 'canceled' projects as 'failed' by replacing value 'canceled' with 'failed'
    - Keep only rows where state is 'failed' or 'successful' and drop all others

    Parameters
    ----------
    df : pd.DataFrame
        Input DataFrame containing a 'state' column.

    Returns
    -------
    df : pd.DataFrame
        A new DataFrame with:
        - 'canceled' merged into 'failed'
        - only 'failed' and 'successful' projects retained
    """
    df = df.copy()

    # Merge 'canceled' with 'failed'
    df["state"] = df["state"].replace({"canceled": "failed"})

    # Keep only failed and successful
    df = df[df["state"].isin(["failed", "successful"])]

    return df


In [12]:
df_p = df.copy()

# pipeline

df_p = fix_dates(df_p)
df_p = calculate_usd_goal(df_p)
df_p = merge_categories(df_p)
df_p = merge_comics(df_p)
df_p = clean_state(df_p)
df_p = drop_unused_columns(df_p)

In [13]:
df_p.isnull().sum()

blurb                    39
category_parent_id        0
category_parent_name      0
country                   0
created_at                0
creator_id                0
currency                  0
deadline                  0
goal                      0
launched_at               0
name                      4
state                     0
launched_date             0
fx_date                   0
fx_daily_mean           180
usd_goal_fx             180
dtype: int64

In [14]:
df_p

,blurb,category_parent_id,category_parent_name,country,created_at,creator_id,currency,deadline,goal,launched_at,name,state,launched_date,fx_date,fx_daily_mean,usd_goal_fx
0,A Year of Sanderson: Enjoy books and swag boxe...,18,Publishing,US,1641845766,74501917,USD,1648767600,1000000.0,1646150407,Surprise! Four Secret Novels by Brandon Sanderson,successful,2022-03-01,2022-03-01,1.00000,1000000.0
1,Color e-paper smartwatch with up to 7 days of ...,7,Design,US,1423542328,597507018,USD,1427508000,500000.0,1424789082,"Pebble Time - Awesome Smartwatch, No Compromises",successful,2015-02-24,2015-02-24,1.00000,500000.0
2,Beginning with The Stormlight Archive and expa...,12,Games,US,1715376496,237961243,USD,1724986800,250000.0,1722949229,Brandon Sanderson's Cosmere® RPG,successful,2024-08-06,2024-08-06,1.00000,250000.0
3,The COOLEST is a portable party disguised as a...,7,Design,US,1382478674,203090294,USD,1409360400,50000.0,1404807277,COOLEST COOLER: 21st Century Cooler that's Act...,successful,2014-07-08,2014-07-08,1.00000,50000.0
4,Euro-inspired dungeon crawling sequel to the 2...,12,Games,US,1581299604,1350948450,USD,1588366800,500000.0,1585667038,Frosthaven,successful,2020-03-31,2020-03-31,1.00000,500000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621885,These are Rockstar style designer tee shirts e...,9,Fashion,SG,1482468515,1533136035,SGD,1485486150,25000.0,1482894150,Black Rose Co. designer tee shirts,failed,2016-12-28,2016-12-28,0.68962,17240.0
621886,Global warming become a serious problem in tod...,9,Fashion,SG,1507792281,1821636636,SGD,1513923648,20000.0,1511331648,Let's Have a Modular Fashion,failed,2017-11-22,2017-11-22,0.74046,14809.0
621887,The world's first modular LEGO-like hydroponic...,10,Food,SG,1605193318,1118131004,SGD,1611733147,39000.0,1606549147,One Kind Block (Relaunched) (Canceled),failed,2020-11-28,2020-11-27,0.74735,29147.0
621888,I would like to set up a highend jewelry brand...,9,Fashion,SG,1502166623,1871765483,SGD,1504964633,5000.0,1502372633,Jewerly brand in Monaco,failed,2017-08-10,2017-08-10,0.73341,3667.0


In [15]:
df_p["currency"].value_counts()

currency
USD    416940
GBP     61144
EUR     48157
CAD     27169
AUD     13885
MXN      7286
HKD      5931
SEK      3221
SGD      2681
JPY      2630
NZD      2419
DKK      1937
CHF      1742
NOK      1032
PLN       400
Name: count, dtype: int64

In [16]:
df_p[df_p["usd_goal_fx"] == 0]

,blurb,category_parent_id,category_parent_name,country,created_at,creator_id,currency,deadline,goal,launched_at,name,state,launched_date,fx_date,fx_daily_mean,usd_goal_fx
319264,$1 a square inch in Detroit + super fun excite...,1,Art,US,1259129231,211945026,USD,1259906820,0.01,1259132089,LOVELAND Round 6: A Force More Powerful,successful,2009-11-25,2009-11-25,1.0,0.0
408155,"Self-published in 2010, """"""""Corruptions, A Nov...",18,Publishing,US,1320547530,388384107,USD,1323791212,0.01,1320680812,"Word-of-mouth publishing: get """"""""Corruptions""...",failed,2011-11-07,2011-11-07,1.0,0.0
414827,True immigrant story,11,Film & Video,US,1327464372,1600537964,USD,1331875399,0.15,1327472599,Nana,failed,2012-01-25,2012-01-25,1.0,0.0
429436,Roxanne’Attempts to save herself from a life s...,11,Film & Video,US,1309942099,1998423069,USD,1311083979,0.50,1310479179,RocknRoll NoisePollution,failed,2011-07-12,2011-07-12,1.0,0.0


In [17]:
df_p[df_p["name"].isnull()]

,blurb,category_parent_id,category_parent_name,country,created_at,creator_id,currency,deadline,goal,launched_at,name,state,launched_date,fx_date,fx_daily_mean,usd_goal_fx
113735,Dive into nanotech infested anomalies in this ...,12,Games,US,1676259821,553653704,USD,1678201200,3500.0,1677596407,NaN,successful,2023-02-28,2023-02-28,1.00000,3500.0
291339,NaN,1,Art,US,1320612894,1751437608,USD,1323061140,35000.0,1320620155,NaN,failed,2011-11-06,2011-11-04,1.00000,35000.0
318871,NaN,11,Film & Video,US,1325021401,887042339,USD,1330524240,200000.0,1325417731,NaN,failed,2012-01-01,2011-12-30,1.00000,200000.0
565050,...,12,Games,GB,1355858653,177117814,GBP,1357509600,2000.0,1355957868,NaN,failed,2012-12-19,2012-12-19,1.62605,3252.0
